[source](https://www.mathworks.com/help/optim/ug/sudoku-puzzles-problem-based.html)

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB
import highspy

from utils.timer import Timer

from numba import njit, jit

from utils.constraintUtils import *
from utils.formattingUtils import *
from utils.constructILPUtils import *
from utils.solveILPUtils import *


In [17]:
@njit
def kron_sum(a,b):
	return (a[:,None] + b[None,:]).flatten()

In [18]:
kron_sum(np.arange(0,4), np.arange(0,40,10))

array([ 0, 10, 20, 30,  1, 11, 21, 31,  2, 12, 22, 32,  3, 13, 23, 33])

In [21]:
@njit
def kron_sum(a,b):
	return (a.reshape(-1, 1) + b.reshape(1, -1)).flatten()

In [22]:
kron_sum(np.arange(0,4), np.arange(0,40,10))

array([ 0, 10, 20, 30,  1, 11, 21, 31,  2, 12, 22, 32,  3, 13, 23, 33])

In [2]:
#The sudoku from the problem
empty_sudoku = np.array(
   [[0, 0, 1, 0, 0, 0, 0, 0, 2],
    [0, 0, 0, 3, 0, 4, 5, 0, 0],
    [6, 0, 0, 7, 0, 0, 1, 0, 0],
    [0, 4, 0, 5, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 8, 0],
    [0, 0, 0, 0, 0, 6, 0, 9, 0],
    [0, 0, 5, 0, 0, 9, 0, 0, 4],
    [0, 0, 8, 2, 0, 1, 0, 0, 0],
    [3, 0, 0, 0, 0, 0, 7, 0, 0]]
)

#a sample sudoku from the internet
sample_solution = np.array(
    [[7, 1, 3, 5, 2, 4, 6, 9, 8],
     [5, 2, 9, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#the same sample sudoku with some elements deleted
empty_sample_solution = np.array(
    [[0, 0, 0, 5, 2, 4, 6, 9, 8],
     [5, 2, 0, 6, 1, 8, 3, 4, 7],
     [6, 4, 8, 7, 3, 9,	2, 5, 1],
     [1, 5, 2, 9, 4, 7, 8, 3, 6],
     [8, 3, 6, 1, 5, 2, 9, 7, 4],
     [4, 9, 7, 3, 8, 6, 5, 1, 2],
     [3, 8, 5, 4, 6, 1, 7, 2, 9],
     [9, 6,	1, 2, 7, 5, 4, 8, 3],
     [2, 7, 4, 8, 9, 3, 1, 6, 5]]
)

#A very empty sudoku
very_empty_clues = np.zeros((9,9), dtype = int)
very_empty_clues[0,1] = 3
very_empty_clues[0,2] = 2
very_empty_clues[0,3] = 1

In [3]:
bin_sample_solution = array_to_vector(sample_solution)
vector_to_array(bin_sample_solution)

array([[7, 1, 3, 5, 2, 4, 6, 9, 8],
       [5, 2, 9, 6, 1, 8, 3, 4, 7],
       [6, 4, 8, 7, 3, 9, 2, 5, 1],
       [1, 5, 2, 9, 4, 7, 8, 3, 6],
       [8, 3, 6, 1, 5, 2, 9, 7, 4],
       [4, 9, 7, 3, 8, 6, 5, 1, 2],
       [3, 8, 5, 4, 6, 1, 7, 2, 9],
       [9, 6, 1, 2, 7, 5, 4, 8, 3],
       [2, 7, 4, 8, 9, 3, 1, 6, 5]])

In [4]:
for i in range(10):

    with Timer("Construnct ILP"):
        c, A_full_sparse = constuct_A_sparse_and_c(empty_sudoku)
        
    with Timer("Solve With Scipy"):
        solve_sudoku_scipy(c, A_full_sparse)

In [5]:
print(Timer.timers)

{'Construnct ILP': [3.410415271995589, 0.001855263952165842, 0.0012171929702162743, 0.001215510070323944, 0.0012157168239355087, 0.0012065209448337555, 0.001226307824254036, 0.0012119091115891933, 0.001204789848998189, 0.001217735931277275], 'Solve With Scipy': [0.04845144087448716, 0.0064667749684304, 0.006328934105113149, 0.006302579073235393, 0.0063275001011788845, 0.006320745917037129, 0.006333310855552554, 0.006360648898407817, 0.006415913812816143, 0.006381043000146747]}


In [6]:
raise Error

NameError: name 'Error' is not defined

In [ ]:
with Timer("njitted"):
    setup_ILP_njit(empty_sudoku)

with Timer("normal"):
    setup_ILP_dense(empty_sudoku)

print(Timer.timers)

In [ ]:
with Timer("Solve Highs"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit)
    #model = solve_sudoku_pisciopt(c, A_full_njit)
    solution = solve_sudoku_highspy(c, A_full_njit_sparse)

print(vector_to_array(solution))
print(Timer.timers)

In [ ]:
#fastest option
with Timer("Solve sudoku"):
    c, A_full_njit = setup_ILP_njit(empty_sudoku)
    A_sparse_njit =  sp.sparse.csr_matrix(A_full_njit)
    res_scipy = solve_sudoku_scipy(c, A_sparse_njit)
    

print(Timer.timers)
    
    

In [ ]:
#fastest option

c, A_full_njit = setup_ILP_njit(empty_sudoku)

for i in range(10):
    A_sparse_njit =  sp.sparse.csr_matrix(A_full_njit)
    with Timer("CSR"):
        solve_sudoku_scipy(c, A_sparse_njit)
    
    A_sparse_njit =  sp.sparse.csc_matrix(A_full_njit)
    with Timer("CSC"):
        solve_sudoku_scipy(c, A_sparse_njit)
    

print(Timer.timers)
    

In [ ]:
repeats = 8
for i in range(repeats):
    with Timer("1. Setup njit dense"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
    with Timer("2. Scipy solver njit dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_dense)
    with Timer("3. Gurobi solver njit dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_dense )
    del c
    del A_full_njit_dense 

    with Timer("1. Setup njit sparse"):
        c, A_full_njit_dense = setup_ILP_njit(empty_sudoku)
        A_full_njit_sparse = sp.sparse.csc_matrix(A_full_njit_dense)
    with Timer("2. Scipy solver njit sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_njit_sparse )
    with Timer("3. Gurobi solver njit sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_njit_sparse )
    with Timer("4. Highspy solver njit sparse"):
        res_highspy = solve_sudoku_highspy(c, A_full_njit_sparse )
    del c
    del A_full_njit_sparse
        
    with Timer("1. Setup sparse_dense"):
        c, A_full_sparse = setup_ILP_sparse_end(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse_dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse_dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup sparse"):
        c, A_full_sparse = setup_ILP_sparse(empty_sudoku, format = "csc", dtype = bool)
    with Timer("2. Scipy solver sparse"):
        res_scipy = solve_sudoku_scipy(c, A_full_sparse)
    with Timer("3. Gurobi solver sparse"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_sparse)
    del c
    del A_full_sparse
    
    with Timer("1. Setup dense"):
        c, A_full_dense = setup_ILP_dense(empty_sudoku, dtype = bool)
    with Timer("2. Scipy solver dense"):
        res_scipy = solve_sudoku_scipy(c, A_full_dense)
    with Timer("3. Gurobi solver dense"):
        res_gurobi = solve_sudoku_gurobi(c, A_full_dense)
    del c
    del A_full_dense

np.set_printoptions(precision=5)
keys = np.sort(list(Timer.timers.keys()))
for key in keys:
    print(key, "\t\t", np.array(Timer.timers[key]))